# Подготовка модели распознавания рукописных букв и цифр

In [3]:
import os
import pickle
import emnist
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn
import cv2
import torch
from torch import nn
from torchvision.transforms import ToTensor, Compose, Resize, Normalize
from torchvision.datasets import EMNIST
from torch.utils.data import DataLoader
from PIL import Image
from torchinfo import summary

In [5]:
def converting (symbol):
    if symbol < 10:
        symbol = symbol + 48
    elif symbol < 36:
        symbol = symbol + 55
    elif symbol < 38: 
        symbol = symbol + 61
    elif symbol < 43: 
        symbol = symbol + 62
    elif symbol < 44: 
        symbol = symbol + 67
    elif symbol < 46: 
        symbol = symbol + 69
    else:
        symbol = symbol + 70

    return chr(symbol)

In [263]:
dataset = EMNIST('data/', 'balanced', download=False)
dataset

Dataset EMNIST
    Number of datapoints: 112800
    Root location: data/
    Split: Train

In [267]:
for i in range(10):
    img, lbl = dataset[i]
    print(converting(lbl), img.size)
    display(img.transpose(method=Image.TRANSPOSE))

r (28, 28)


a (28, 28)


n (28, 28)


F (28, 28)


4 (28, 28)


h (28, 28)


Q (28, 28)


W (28, 28)


K (28, 28)


1 (28, 28)


# 1 version

Попробуем простую модель из одного линейного слоя

In [7]:
transform = Compose([
    ToTensor(),
    Normalize([0.5], [0.5])
])

train_dataset = EMNIST('data/', 'balanced', download=False, transform=transform)
val_dataset = EMNIST('data/', 'balanced', download=False, train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1000, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1000)

In [31]:
class LogReg(nn.Module):
    def __init__(self, in_features, n_classes):
        super(LogReg, self).__init__()
        self.fc = nn.Linear(in_features, n_classes)

    def forward(self, x):
        return self.fc(x)

In [50]:
model = LogReg(in_features=28*28, n_classes=47)
loss_f = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

n_epoch = 50
val_fre = 10

model.train()
for epoch in range(n_epoch):
    loss_sum = 0
    for step, (data, target) in enumerate(train_loader):
        data = data.flatten(start_dim=1)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_f(output, target)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()

    print(f'Epoch: {epoch} \tLoss: {loss_sum / (step + 1):.6f}')

    if (epoch+1) % val_fre == 0:
        model.eval()
        loss_sum = 0
        correct = 0
        for step, (data, target) in enumerate(val_loader):
            data = data.flatten(start_dim=1)
            with torch.no_grad():
                output = model(data)
                loss = loss_f(output, target)
            loss_sum += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
        acc = correct / len(val_loader.dataset)
        print(f'Val Loss: {loss_sum / (step + 1):.6f} \tAccuracy: {acc}')
        model.train()

Epoch: 0 	Loss: 2.051472
Epoch: 1 	Loss: 1.443084
Epoch: 2 	Loss: 1.331690
Epoch: 3 	Loss: 1.274793
Epoch: 4 	Loss: 1.239362
Epoch: 5 	Loss: 1.215182
Epoch: 6 	Loss: 1.195533
Epoch: 7 	Loss: 1.182575
Epoch: 8 	Loss: 1.170163
Epoch: 9 	Loss: 1.160444
Val Loss: 1.199963 	Accuracy: 0.6693085106382979
Epoch: 10 	Loss: 1.152815
Epoch: 11 	Loss: 1.145763
Epoch: 12 	Loss: 1.138802
Epoch: 13 	Loss: 1.134246
Epoch: 14 	Loss: 1.128831
Epoch: 15 	Loss: 1.123492
Epoch: 16 	Loss: 1.120272
Epoch: 17 	Loss: 1.117231
Epoch: 18 	Loss: 1.113529
Epoch: 19 	Loss: 1.110341
Val Loss: 1.163621 	Accuracy: 0.6823936170212765
Epoch: 20 	Loss: 1.107762
Epoch: 21 	Loss: 1.104937
Epoch: 22 	Loss: 1.101991
Epoch: 23 	Loss: 1.100299
Epoch: 24 	Loss: 1.098150
Epoch: 25 	Loss: 1.095890
Epoch: 26 	Loss: 1.093700
Epoch: 27 	Loss: 1.092410
Epoch: 28 	Loss: 1.091308
Epoch: 29 	Loss: 1.089684
Val Loss: 1.149731 	Accuracy: 0.6848404255319149
Epoch: 30 	Loss: 1.087789
Epoch: 31 	Loss: 1.086435
Epoch: 32 	Loss: 1.084968
Epoch

Точность модели недостаточная

# 2 version

Добавим второй линейный слой

In [33]:
def train(model, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre):
    model.train()
    for epoch in range(n_epoch):
        loss_sum = 0
        for step, (data, target) in enumerate(train_loader):
            data = data.flatten(start_dim=1)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_f(output, target)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()

        print(f'Epoch: {epoch} \tLoss: {loss_sum / (step + 1):.6f}')

        if epoch % val_fre == 0:
            validate(model, val_loader)

def validate(model, val_loader):
    model.eval()
    loss_sum = 0
    correct = 0
    for step, (data, target) in enumerate(val_loader):
        data = data.flatten(start_dim=1)
        with torch.no_grad():
            output = model(data)
            loss = loss_f(output, target)
        loss_sum += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    acc = correct / len(val_loader.dataset)
    print(f'Val Loss: {loss_sum / (step + 1):.6f} \tAccuracy: {acc}')
    model.train()

class MLP(nn.Module):
    def __init__(self, in_features, hid_features, n_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(in_features, hid_features)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hid_features, n_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model_mlp = MLP(in_features=28*28, hid_features=2^47, n_classes=47)
print(model_mlp)
loss_f = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_mlp.parameters(), lr=1e-1)

n_epoch = 20
val_fre = 10

train(model_mlp, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre)
validate(model_mlp, val_loader)

MLP(
  (fc1): Linear(in_features=784, out_features=45, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=45, out_features=47, bias=True)
)
Epoch: 0 	Loss: 2.607973
Val Loss: 1.720239 	Accuracy: 0.5463829787234042
Epoch: 1 	Loss: 1.507018
Epoch: 2 	Loss: 1.322468
Epoch: 3 	Loss: 1.242596
Epoch: 4 	Loss: 1.188235
Epoch: 5 	Loss: 1.144420
Epoch: 6 	Loss: 1.107208
Epoch: 7 	Loss: 1.072965
Epoch: 8 	Loss: 1.040199
Epoch: 9 	Loss: 1.011349
Epoch: 10 	Loss: 0.981901
Val Loss: 1.005871 	Accuracy: 0.7139893617021277
Epoch: 11 	Loss: 0.956062
Epoch: 12 	Loss: 0.929783
Epoch: 13 	Loss: 0.908262
Epoch: 14 	Loss: 0.885797
Epoch: 15 	Loss: 0.864728
Epoch: 16 	Loss: 0.847744
Epoch: 17 	Loss: 0.829077
Epoch: 18 	Loss: 0.815442
Epoch: 19 	Loss: 0.800696
Val Loss: 0.850914 	Accuracy: 0.7526595744680851


Результат получился значительно лучше, но цель недостигнута

# 3 version

Добавим сверточные слои

In [105]:


def train(model, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre):
    model.train()
    for epoch in range(n_epoch):
        loss_sum = 0
        for step, (data, target) in enumerate(train_loader):
            
            optimizer.zero_grad()
            output = model(data).squeeze(1)
            
            loss = loss_f(output, target)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()

        print(f'Epoch: {epoch} \tLoss: {loss_sum / (step + 1):.6f}')

        if epoch % val_fre == 0:
            validate(model, val_loader)

def validate(model, val_loader):
    model.eval()
    loss_sum = 0
    correct = 0
    for step, (data, target) in enumerate(val_loader):
        
        with torch.no_grad():
            output = model(data).squeeze(1)
            loss = loss_f(output, target)
        loss_sum += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    acc = correct / len(val_loader.dataset)
    print(f'Val Loss: {loss_sum / (step + 1):.6f} \tAccuracy: {acc}')
    model.train()



class MLP(nn.Module):
    def __init__(self, hid_features, n_classes):
        super(MLP, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 1)
        self.conv2 = nn.Conv2d(32, 64, 1)
        
        self.fc1 = nn.Linear(64*7*7, hid_features)
        self.relu = nn.ReLU()
        
        self.fc2 = nn.Linear(hid_features, n_classes)

    def forward(self, x):
       
        x = self.conv1(x)
        x = self.relu(x)
        x = nn.functional.max_pool2d(x, 2)
        x = self.conv2(x)
        x = self.relu(x)
        x = nn.functional.max_pool2d(x, 2)   
        x = x.view(-1, 64 * 7 * 7)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model_mlp = MLP( hid_features=2^47, n_classes=47)
print(model_mlp)
loss_f = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_mlp.parameters(), lr=1e-1)

n_epoch = 20
val_fre = 10

train(model_mlp, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre)
validate(model_mlp, val_loader)

MLP(
  (conv1): Conv2d(1, 32, kernel_size=(1, 1), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Linear(in_features=3136, out_features=45, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=45, out_features=47, bias=True)
)
Epoch: 0 	Loss: 3.584606
Val Loss: 2.711091 	Accuracy: 0.2873404255319149
Epoch: 1 	Loss: 1.930190
Epoch: 2 	Loss: 1.455408
Epoch: 3 	Loss: 1.314292
Epoch: 4 	Loss: 1.228303
Epoch: 5 	Loss: 1.162551
Epoch: 6 	Loss: 1.109569
Epoch: 7 	Loss: 1.067838
Epoch: 8 	Loss: 1.040284
Epoch: 9 	Loss: 1.000452
Epoch: 10 	Loss: 0.971190
Val Loss: 0.976999 	Accuracy: 0.7079255319148936
Epoch: 11 	Loss: 0.942609
Epoch: 12 	Loss: 0.924762
Epoch: 13 	Loss: 0.912016
Epoch: 14 	Loss: 0.887587
Epoch: 15 	Loss: 0.871460
Epoch: 16 	Loss: 0.857944
Epoch: 17 	Loss: 0.847320
Epoch: 18 	Loss: 0.839078
Epoch: 19 	Loss: 0.828477
Val Loss: 0.869244 	Accuracy: 0.7370212765957447


Результат улучшить не удалось

# 4 Version

Добавим батч-нормализацию и макспулинг

In [110]:
def train(model, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre):
    model.train()
    for epoch in range(n_epoch):
        loss_sum = 0
        for step, (data, target) in enumerate(train_loader):
            
            optimizer.zero_grad()
            output = model(data).squeeze(1)
            
            loss = loss_f(output, target)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()

        print(f'Epoch: {epoch} \tLoss: {loss_sum / (step + 1):.6f}')

        if epoch % val_fre == 0:
            validate(model, val_loader)

def validate(model, val_loader):
    model.eval()
    loss_sum = 0
    correct = 0
    for step, (data, target) in enumerate(val_loader):
        
        with torch.no_grad():
            output = model(data).squeeze(1)
            loss = loss_f(output, target)
        loss_sum += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    acc = correct / len(val_loader.dataset)
    print(f'Val Loss: {loss_sum / (step + 1):.6f} \tAccuracy: {acc}')
    model.train()

class MLP(nn.Module):
    
    def __init__(self):
        super(MLP, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=3000)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=3000, out_features=600)
        self.fc3 = nn.Linear(in_features=600, out_features=47)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

model_mlp = MLP()
print(model_mlp)
loss_f = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_mlp.parameters(), lr=1e-1)

n_epoch = 20
val_fre = 10

train(model_mlp, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre)
validate(model_mlp, val_loader)

MLP(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=3000, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=3000, out_features=600, bias=True)
  (fc3): Linear(in_features=600, out_features=47, bias=True)
)


C:\Users\Msi-1\anaconda3\Lib\site-packages\torch\nn\functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch: 0 	Loss: 1.162711
Val Loss: 0.654232 	Accuracy: 0.7924468085106383
Epoch: 1 	Loss: 0.560543
Epoch: 2 	Loss: 0.488813
Epoch: 3 	Loss: 0.454339
Epoch: 4 	Loss: 0.436435
Epoch: 5 	Loss: 0.412805
Epoch: 6 	Loss: 0.396634
Epoch: 7 	Loss: 0.382840
Epoch: 8 	Loss: 0.372654
Epoch: 9 	Loss: 0.365446
Epoch: 10 	Loss: 0.355684
Val Loss: 0.408625 	Accuracy: 0.8633510638297872
Epoch: 11 	Loss: 0.347033
Epoch: 12 	Loss: 0.340145
Epoch: 13 	Loss: 0.334208
Epoch: 14 	Loss: 0.326531
Epoch: 15 	Loss: 0.321561
Epoch: 16 	Loss: 0.315878
Epoch: 17 	Loss: 0.307617
Epoch: 18 	Loss: 0.305559
Epoch: 19 	Loss: 0.298119
Val Loss: 0.378924 	Accuracy: 0.8720212765957447


Результат достиг пороговой отметки

# 5 Version

Добавим еще один сверточный слой и поэкспериментируем с промежуточными фичами

In [288]:
def train(model, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre):
    model.train()
    for epoch in range(n_epoch):
        loss_sum = 0
        for step, (data, target) in enumerate(train_loader):
            
            optimizer.zero_grad()
            output = model(data).squeeze(1)
            
            loss = loss_f(output, target)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()

        print(f'Epoch: {epoch} \tLoss: {loss_sum / (step + 1):.6f}')

        if epoch % val_fre == 0:
            validate(model, val_loader)

def validate(model, val_loader):
    model.eval()
    loss_sum = 0
    correct = 0
    for step, (data, target) in enumerate(val_loader):
        
        with torch.no_grad():
            output = model(data).squeeze(1)
            loss = loss_f(output, target)
        loss_sum += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    acc = correct / len(val_loader.dataset)
    print(f'Val Loss: {loss_sum / (step + 1):.6f} \tAccuracy: {acc}')
    model.train()

class MMM(nn.Module):
    
    def __init__(self):
        super(MMM, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=128*2*2, out_features=6000)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=6000, out_features=1200)
        self.fc3 = nn.Linear(in_features=1200, out_features=400)
        self.fc4 = nn.Linear(in_features=400, out_features=47)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
       
        out = out.view(out.size(0), -1)
       
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.drop(out)
        out = self.fc3(out)
        out = self.fc4(out)
        
        return out

model_mmm = MMM()
print(model_mmm)
loss_f = nn.CrossEntropyLoss()
optimizer_mmm = torch.optim.SGD(model_mmm.parameters(), lr=1e-1)

n_epoch = 20
val_fre = 10

train(model_mmm, optimizer_mmm, loss_f, train_loader, val_loader, n_epoch, val_fre)
validate(model_mmm, val_loader)

MMM(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=512, out_features=6000, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=6000, out_featur

Результат значительно улучшить не удалось, но метрика все еще выше порога - оставляем такой вариант

In [298]:
torch.save(model_mmm.state_dict(), 'cnn.ckpt')

